In [2]:
import jax.numpy as jnp
import jax

keys = jax.random.split(jax.random.PRNGKey(0), 10)
nn = 84
weights = jax.random.normal (keys[0], (nn, nn))
lengths = jax.random.randint(keys[1], (nn, nn), 0, 255)
buffer = jax.random.normal(keys[2], (nn, lengths.max()+1))

Not done yet: how to register `vjp` as the VJP for fwd?

In [5]:
def make_conn(weights, lengths):
    nn = weights.shape[0]
    ns = jnp.tile(jnp.arange(nn),(nn,1))
    
    def fwd(buffer):
        return jnp.sum(weights*buffer[ns,lengths], axis=1)

    def vjp(g,buffer):
        gb = jnp.tile(g[:,None],(1,nn)) # rev of sum(a, axis=1)
        return jnp.zeros_like(buffer).at[ns,lengths.T].add(weights.T*gb)
    
    return fwd, vjp

f = jax.jit(make_conn(weights, lengths))
f(buffer)
%timeit f(buffer)

11.2 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
gl = jax.jit(jax.grad(lambda b: jnp.sum(jnp.square(f(b + 1)))))
gl(buffer)
%timeit gl(buffer + 1).block_until_ready()

63.3 µs ± 6.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Well that doesn't seem to be too slow.